# Simulation of NBR07 with measured parameters

In [1]:
from ResSimulator import NBResonator

In [2]:
from TrappingSimulator import QPtrapper

In [3]:
savepath = r"G:\Shared drives\LFL\Simulations\James\NBR07\\"

In [4]:
import os
if not os.path.exists(savepath):
    os.makedirs(savepath)

## definition of parameters used in QP trapper

In [5]:
duration = 0.05 # seconds to record data
sampleRate = 300e6
N = int(duration*sampleRate)
tauTrap = 40e-6
tauRelease = 10e-6
tauCommon = 1e-4
tauRare = 7e-1
tauRecomb = 1.5e-4
phi = 0.45
Lj = 20.8475e-12 # squid inductance at zero phase bias
# Lj = 31.7299e-12
args = {'N':N,'Lj':Lj,'tauTrap':tauTrap,'tauRelease':tauRelease,'tauCommon':tauCommon,'tauRare':tauRare,
        'tauRecomb':tauRecomb,'sampleRate':sampleRate,'phi':phi,'Delta':2.72370016e-23,'T':0.010}

In [6]:
trapper = QPtrapper(**args)

## Now we have the trapping events, let's generate the resonator response. define some more parameters

In [7]:
L = 1.82897e-9
C = 0.739929e-12
# L = 2.7837e-9
# C = 0.486155e-12
Qi = 53000
Qe = 25000
photonRO = 45
photonNoise = 3
delKappa = -0.5

resArgs = {'L':L,'C':C,'photonRO':photonRO,'photonNoise':photonNoise,'Qi':Qi,'Qe':Qe,'sampleRate':sampleRate,'delKappa':delKappa}

res = NBResonator(trapper,**resArgs)

In [8]:
res.dParams

{'fd': 4277461605.8818517,
 'f0': 4277587512.2312684,
 'Qt': 16987.17948717949,
 'Qi': 53000,
 'Qe': 25000,
 'N': 15000000,
 'q': 0.02241579930274248,
 'photonRO': 45,
 'sampleRate': 300000000.0,
 'kappa': 1582185.8494702321,
 'fwhm': 251812.6988332369,
 'diameter': 1.358974358974359,
 'freq_shift': 192235.2624739662,
 'SNR': 0.024948213529382368,
 'SNRdB': -16.029605475199105,
 'sigma': 6.3311160427631625}

## start some analysis with integration for desired SNR

In [9]:
from scipy.constants import pi
from scipy.signal import windows, convolve
avgTime5 = 4*res.dParams['Qt']*5/(res.dParams['photonRO']*2*pi*res.dParams['f0'])
avgTime10 = 4*res.dParams['Qt']*10/(res.dParams['photonRO']*2*pi*res.dParams['f0'])
avgTime25 = 4*res.dParams['Qt']*25/(res.dParams['photonRO']*2*pi*res.dParams['f0'])
avgTime50 = 4*res.dParams['Qt']*50/(res.dParams['photonRO']*2*pi*res.dParams['f0'])
print('integrate for {} us for RF power SNR=5'.format(avgTime5*1e6))
print('integrate for {} us for RF power SNR=50'.format(avgTime50*1e6))

integrate for 0.2809053339677251 us for RF power SNR=5
integrate for 2.809053339677251 us for RF power SNR=50


### perform convolution for 5 microseconds

In [10]:
nAvg = int(max(2e-6*res.dParams['sampleRate'],1))
window = windows.hann(nAvg)
# rhann = convolve(res.signal.real,window,mode='same')/sum(window)
# ihann = convolve(res.signal.imag,window,mode='same')/sum(window)
rhann = np.mean(res.signal.real[:len(res.signal)//(nAvg)*nAvg].reshape((len(res.signal)//(nAvg),nAvg)),axis=1)
ihann = np.mean(res.signal.imag[:len(res.signal)//(nAvg)*nAvg].reshape((len(res.signal)//(nAvg),nAvg)),axis=1)

NameError: name 'np' is not defined

### Plot a segment as time series for visual

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
time = np.arange(res.dParams['N'])/res.dParams['sampleRate']
time = np.mean(time[:len(res.signal)//(nAvg)*nAvg].reshape((len(res.signal)//(nAvg),nAvg)),axis=1)

h = plt.subplot()
pltmask = np.logical_and(time > 3e-3, time < 3.5e-3)
plttime = time[pltmask]*1e6 - 3000
h.plot(plttime,rhann[pltmask],label='I(t)')
h.plot(plttime,ihann[pltmask],color='yellow',label='Q(t)')
h.set_xlabel('Time [$\mu$s]',fontsize = 16)
h.set_ylabel('V / |V|',fontsize = 14)
h.tick_params(labelsize=12)
h.legend(fontsize=12)
plt.savefig(savepath+r'NBR07_noisySignal_2microsecond.png',format='png')
plt.show()

### plot complex histogram

In [ ]:
from matplotlib.patches import Ellipse
from matplotlib.colors import LogNorm

In [ ]:
h = plt.subplot()
hs = plt.hist2d(rhann,ihann,bins=(80,80),norm=LogNorm(),cmap=plt.get_cmap('Greys'))
hb = plt.colorbar(hs[-1], shrink=0.9, extend='both')
h.set_aspect('equal')
h.grid()
h.set_xlabel('I',fontsize=16)
h.set_ylabel('Q',fontsize=14)
h.tick_params(labelsize=12)
plt.savefig(savepath+r'NBR07_hist.png',format='png')
plt.show()

In [ ]:
fig,ax = plt.subplots(2,1,sharex=True,figsize=[9,6],constrained_layout=True)
# ax[0].plot(time*1e3,np.arctan((ihann/rhann)))
# ax[0].set_ylabel('phase [rad]')
# ax[1].plot(time*1e3,np.sqrt(rhann**2 + ihann**2))
# ax[1].set_ylabel('Magnitude')
# ax[1].set_xlabel('time [ms]')
ax[0].plot(plttime,rhann[pltmask], label='I')
ax[0].plot(plttime,ihann[pltmask], color='yellow',label = 'Q')
ax[0].set_title('Simulated - 2 $\mu$s integration')
ax[0].set_ylabel('Quadratures [mV]')
ax[0].legend()
ax[1].plot(plttime,np.abs(rhann[pltmask] + 1j*ihann[pltmask]))
ax[1].set_ylabel('Magnitude [mV]')
ax[1].set_xlabel('Time [$\mu$s]')


plt.savefig(savepath+r'NBR07_mag_phase.png',format='png')
plt.show()

# Is this level of SNR sufficient for machine learning alg to detect occupation?

In [ ]:
avgTime25

### We're already averaging for 18.6 microseconds and assuming quantum limited noise

In [ ]:
print('avg # trapped = {:.4}'.format(np.mean(trapper.nTrapped)))
print('max # = {}'.format(np.max(trapper.nTrapped)))

# use scikit package for expectation-maximization algorithm to fit gaussian mixture

In [ ]:
from sklearn.mixture import GaussianMixture

### initial guess at mode locations, based on looking at complex hist

In [ ]:
means_guess = np.array([[-0.5,0.4,-0.6,-1],[0.75,-0.25,-0.5,0]]).T

## run the ML algorithm

In [ ]:
estimator = GaussianMixture(n_components = len(means_guess),means_init=means_guess)
estimator.fit(np.array([rhann,ihann]).T)

### Did it work?

In [ ]:
estimator.converged_

In [ ]:
estimator.means_

## Make the pretty plot

In [ ]:
from matplotlib.patches import Ellipse
from matplotlib.colors import LogNorm

colors = ['red','orange','yellow','green']
def make_ellipses(gmm,ax):
    for n, color in enumerate(colors):
        # get the covariance matrix for the mode associated with n trapped QPs
        covariances = gmm.covariances_[n][:2,:2]
        # v are the eigenvalues of covariance matrix, aka the variances along major and minor axis of ellipse. w are the eigenvectors. Order is smallest to v to largest v
        v, w = np.linalg.eigh(covariances)
        # normalize the eigenvector associated with the smallest variance, i.e., the variance along the minor axis.
        u = w[0] / np.linalg.norm(w[0])
        # get the angle from +x axis to minor axis of ellipse
        angle = 180*np.arctan2(u[1],u[0])/np.pi
        # v is now the diameter of the ellipse in minor, major order. It is equal to 2 std deviations.
        v = 2. *np.sqrt(v)
        # make the ellipse for mode n. Centered at mean with major and minor radius of 1 std deviation. and rotated to align to the data.
        ell = Ellipse(gmm.means_[n,:2],v[0],v[1],180+angle,color=color)
        # now we just add the ellipses to the plot. Note that these ellipses shade the area in which all data points are within 1 std deviation of the mean.
        ell.set_clip_box(ax.bbox)
        ell.set_alpha(0.9)
        ax.add_artist(ell)
        ax.set_aspect('equal','datalim')

h = plt.subplot()
make_ellipses(estimator,h)
hs = plt.hist2d(rhann,ihann,bins=(80,80),norm=LogNorm(),cmap=plt.get_cmap('Greys'))
hb = plt.colorbar(hs[-1], shrink=0.9, extend='both')
h.set_xlabel('I',fontsize=16)
h.set_ylabel('Q',fontsize=14)
h.tick_params(labelsize=12)
plt.savefig(savepath+r'NBR07_hist_modes.png',format='png')
plt.show()

# Does the estimator accurately predict the trap state?

In [ ]:
nEst = np.empty(res.dParams['N'],dtype=int)
nEst[:] = estimator.predict(np.transpose((rhann,ihann)))

### How do the avg number of trapped QPs compare?

In [ ]:
print('simulated avg # = {:.4}'.format(np.mean(trapper.nTrapped)))
print('estimator avg # = {:.4}'.format(np.mean(nEst)))

### That's a good agreement. How does it look when we plot two segments together?

In [ ]:
h = plt.subplot()
h.plot(time[100000:500000]*1e6,trapper.nTrapped[100000:500000],'r',alpha=0.9,label='Simulation',lw=0.8)
h.plot(time[100000:500000]*1e6,nEst[100000:500000],'b',alpha=0.9,label='Estimated',lw=0.8)
h.set_xlabel('Time [$\mu$s]',fontsize = 16)
h.set_ylabel('Number of trapped QPs',fontsize = 14)
plt.yticks([0,1,2,3,4])
h.tick_params(labelsize=12)
h.legend(fontsize=14)
plt.savefig(savepath+r'NBR07_estimationComparison.png',format='png')
plt.show()

### Looks like the estimator is switching far too often. perhaps a different averaging method will work better (instead of convolution with Hann window)

## Let's quantitatively look at the error

In [ ]:
errTrap = np.diff((nEst,trapper.nTrapped),axis=0)[0]

#### errTrap is the difference, errTrap[i] = nTrapped[i] - nEst[i]. It should be positive when we underestimate and negative when we overestimate. The mean should approach 0 for large dataset unless we are somehow more prone to over or under estimating.

In [ ]:
np.mean(errTrap)

##### Ok, now let's quantify the error. The root mean square of the difference should tell us what percent of the time we're wrong about the state.

In [ ]:
np.sqrt(np.mean(errTrap**2))*100 

#### Being wrong 40% of the time isn't exactly something to brag about..

### Actually, the above isn't really how often we're wrong, but overestimates this since errTrap is the difference and can be greater than zero. what we need is an array that is 0 when the estimator is right and 1 when it is wrong. mean of this is the actual percent of time we're wrong.

In [ ]:
estFalse = np.array(np.abs(errTrap) > 0,dtype=int)

In [ ]:
np.mean(estFalse)

# Repeat for exponential window

In [ ]:
nAvg = int(max(5e-6*res.dParams['sampleRate'],1))
window = windows.exponential(nAvg,tau=-(nAvg-1)/np.log(0.00001))
rhann = convolve(res.signal.real,window,mode='same')/sum(window)
ihann = convolve(res.signal.imag,window,mode='same')/sum(window)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
time = np.arange(res.dParams['N'])/res.dParams['sampleRate']

h = plt.subplot()
h.plot(time[150000:250000]*1e6,rhann[150000:250000],'r',alpha=0.9,label='I(t)')
h.plot(time[150000:250000]*1e6,ihann[150000:250000],'b',alpha=0.9,label='Q(t)')
h.set_xlabel('Time [$\mu$s]',fontsize = 16)
h.set_ylabel('V / |V|',fontsize = 14)
h.tick_params(labelsize=12)
h.legend(fontsize=12)
plt.savefig(savepath+r'NBR07_noisySignal_SNR25_exp.png',format='png')
plt.show()

In [ ]:
plt.plot(window)

In [ ]:
nAvg

In [ ]:
h = plt.subplot()
hs = plt.hist2d(rhann,ihann,bins=(80,80),norm=LogNorm(),cmap=plt.get_cmap('Greys'))
hb = plt.colorbar(hs[-1], shrink=0.9, extend='both')
h.set_aspect('equal')
h.grid()
h.set_xlabel('I',fontsize=16)
h.set_ylabel('Q',fontsize=14)
h.tick_params(labelsize=12)
plt.savefig(savepath+r'NBR07_hist_exp.png',format='png')
plt.show()

In [ ]:
h,ax = plt.subplots(2,1,sharex=True)
ax[0].plot(time*1e3,np.arctan((ihann/rhann)))
ax[0].set_ylabel('phase [rad]')
ax[1].plot(time*1e3,np.sqrt(rhann**2 + ihann**2))
ax[1].set_ylabel('Magnitude')
ax[1].set_xlabel('time [ms]')
plt.savefig(savepath+r'NBR07_mag_phase_exp.png',format='png')
plt.show()

In [ ]:
estimator = GaussianMixture(n_components = len(means_guess),means_init=means_guess)
estimator.fit(np.array([rhann,ihann]).T)

In [ ]:
estimator = GaussianMixture(n_components = len(means_guess),means_init=means_guess)
estimator.fit(np.array([rhann,ihann]).T)

### Did it work?

estimator.converged_

estimator.means_

In [ ]:
from matplotlib.patches import Ellipse
from matplotlib.colors import LogNorm

colors = ['red','orange','yellow','green']
def make_ellipses(gmm,ax):
    for n, color in enumerate(colors):
        # get the covariance matrix for the mode associated with n trapped QPs
        covariances = gmm.covariances_[n][:2,:2]
        # v are the eigenvalues of covariance matrix, aka the variances along major and minor axis of ellipse. w are the eigenvectors. Order is smallest to v to largest v
        v, w = np.linalg.eigh(covariances)
        # normalize the eigenvector associated with the smallest variance, i.e., the variance along the minor axis.
        u = w[0] / np.linalg.norm(w[0])
        # get the angle from +x axis to minor axis of ellipse
        angle = 180*np.arctan2(u[1],u[0])/np.pi
        # v is now the diameter of the ellipse in minor, major order. It is equal to 2 std deviations.
        v = 2. *np.sqrt(v)
        # make the ellipse for mode n. Centered at mean with major and minor radius of 1 std deviation. and rotated to align to the data.
        ell = Ellipse(gmm.means_[n,:2],v[0],v[1],180+angle,color=color)
        # now we just add the ellipses to the plot. Note that these ellipses shade the area in which all data points are within 1 std deviation of the mean.
        ell.set_clip_box(ax.bbox)
        ell.set_alpha(0.9)
        ax.add_artist(ell)
        ax.set_aspect('equal','datalim')

h = plt.subplot()
make_ellipses(estimator,h)
hs = plt.hist2d(rhann,ihann,bins=(80,80),norm=LogNorm(),cmap=plt.get_cmap('Greys'))
hb = plt.colorbar(hs[-1], shrink=0.9, extend='both')
h.set_xlabel('I',fontsize=16)
h.set_ylabel('Q',fontsize=14)
h.tick_params(labelsize=12)
plt.savefig(savepath+r'NBR07_hist_modes_exp.png',format='png')
plt.show()

In [ ]:
nEst = np.empty(res.dParams['N'],dtype=int)
nEst[:] = estimator.predict(np.transpose((rhann,ihann)))

### How do the avg number of trapped QPs compare?

print('simulated avg # = {:.4}'.format(np.mean(trapper.nTrapped)))
print('estimator avg # = {:.4}'.format(np.mean(nEst)))

### That's a good agreement. How does it look when we plot two segments together?

h = plt.subplot()
h.plot(time[100000:500000]*1e6,trapper.nTrapped[100000:500000],'r',alpha=0.9,label='Simulation',lw=0.8)
h.plot(time[100000:500000]*1e6,nEst[100000:500000],'b',alpha=0.9,label='Estimated',lw=0.8)
h.set_xlabel('Time [$\mu$s]',fontsize = 16)
h.set_ylabel('Number of trapped QPs',fontsize = 14)
plt.yticks([0,1,2,3,4])
h.tick_params(labelsize=12)
h.legend(fontsize=14)
plt.savefig(savepath+r'NBR07_estimationComparison_exp.png',format='png')
plt.show()

In [ ]:
def f_n_phi(phi,n, L = 1.82897e-9,C = 0.739929e-12,Lj = 20.8475e-12):
    de = np.pi*phi
    Ljphi = Lj/(1-np.sin(de/2)*np.arctanh(np.sin(de/2)))
    q = Ljphi/(L+Ljphi)
    Delta=2.72370016e-23
    rphi0 = (2.06783383*1e-15)/(2*np.pi)
    f0 = 1/(2*np.pi*np.sqrt((L+Ljphi)*C))
    alpha = Delta/(2*(rphi0**2))
    L1 = alpha*(np.cos(de)/np.sqrt(1-np.sin(de/2)**2) + (np.sin(de)**2)/(4*(np.sqrt(1-np.sin(de/2)**2)**3)))
    return f0 -  (q*f0*Ljphi*n*L1/2)
#     return L1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x = np.arange(0,0.45,0.01)

In [ ]:
plt.plot(x,f_n_phi(x,0))
plt.plot(x,f_n_phi(x,1))
plt.plot(x,f_n_phi(x,2))
plt.plot(x,f_n_phi(x,3))

In [ ]:
plt.plot(x,np.squeeze(np.diff((f_n_phi(x,1),f_n_phi(x,0)),axis=0)))